# Primera Parte

In [1]:
# Importamos las librerias

import numpy as np 
import pandas as pd
import re

# Me sale un warning de que estoy borrando elementos a veces y tengo que ejecutarlo dos veces para que funcione.
pd.set_option('mode.chained_assignment', None)
# Abrimos el documento csv

complete_data = pd.read_csv('GSAF5.csv', engine='python')
data = complete_data.copy()

# Elimino las columnas innecesarias

data.drop(['Unnamed: 22','Unnamed: 23','pdf','href formula','Case Number','href','Case Number.1','Case Number.2','original order','Investigator or Source'], inplace = True, axis = 1)


# Renombrar las columnas que continen espacios

data.rename(columns = {'Species ': 'Species', 'Sex ': 'Sex'}, inplace = True)

# Elimino los duplicados que hay.

data = data.drop_duplicates()

data.head()


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN
1,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN
2,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN
3,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN
4,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark


In [2]:
# Cambio los NaN por Unkown

for i in data.columns:
    data[i] = data[i].fillna('Unkown')

In [3]:
# Compruebo que Date y Year contiene los mismos datos
# Si year es 0, lo reemplazo por la fecha de date
# Creo una lista rows para añadir los indices de las filas que quiero eliminar despues

lst = []
rows = []

    
for i in data.index:
    x = re.findall('\d{4}', data['Date'][i])
    if x == []:
        continue
    if int(data['Year'][i]) == 0:
        data['Year'][i] = int(x[0])
    if int(x[0]) == data['Year'][i]:
        continue 
    lst.append((int(x[0]),data['Year'][i],i))
    rows.append(i)
    
print(f"Estos datos son erroneos y sus filas: {lst}")

# Hay algunos datos erroneos que no tienen los mismos valores

# Aqui decido borrar los datos erroneos

data.drop(rows,inplace = True)

Estos datos son erroneos y sus filas: [(2010, 2011, 717), (1996, 1966, 3371), (2176, 1959, 3852), (1995, 1954, 4102), (1923, 1922, 4983), (1929, 1921, 5012), (1893, 1892, 5441), (1878, 1877, 5612), (1870, 1875, 5626), (1888, 1868, 5669), (1850, 1852, 5750), (1820, 1825, 5816), (1600, 1642, 5855)]


In [4]:
# Aqui corrijo algunos datos erroneos y borro otros de la columna 'Sex'

data['Sex'] = data['Sex'].astype(str) 
lst = []
rows = []
for i in data.index:
    x = data['Sex'][i]
    if x == 'M ':
        data['Sex'][i] = 'M'
    elif str(x) == 'nan':
        data['Sex'][i] = 'Unknown'
    elif str(x) not in ('F','M','Unkown'):
        rows.append(i)
        lst.append((x,i))
        
print(f"Los valores erroneos y sus filas son {lst}")  

# Aqui elimino los datos erroneos
data.drop(rows,inplace = True)
data['Sex'].value_counts()

Los valores erroneos y sus filas son [('lli', 1400), ('N', 4708), ('.', 5201)]


M         4825
F          583
Unkown     566
Name: Sex, dtype: int64

In [5]:
# En la columna 'Type' cambio boating por boat 
for i in data.index:
    if data['Type'][i] == 'Boating':
        data['Type'][i] = 'Boat'
        
data['Type'].value_counts()

data.to_csv('Total_Data_Cleaned.csv')

# Segunda Parte


In [6]:
# Me quedo con los datos entre los años 2000 - 2016 y elimino los demas.

data_1 = data.copy()

rows = [i for i in data_1.index if data_1["Year"][i] <= 2000]
        
data_1.drop(rows,inplace = True)

data_1.head()


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,Unkown
1,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,Unkown
2,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,Unkown
3,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,Unkown,Struck by fin on chest & leg,N,Unkown,Unkown
4,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,Unkown,No injury: Knocked off board by shark,N,Unkown,2 m shark


In [7]:
# Aqui elimino algunos datos erroneos en 'date' y 'year' no coinciden

rows = [i for i in data_1.index if not str(data_1['Year'][i]%100) in data_1['Date'][i][-2:]]

data.drop(rows,inplace = True)


In [8]:
# Aqui modifico los elementos de 'Date' para obtenerlos en el formato dd-www-dd
# Los que no lo cumplen los elimino

rows = []

for i in data_1.index:
    x = re.findall("\d{2}-\w{3}-\d{2}",data_1['Date'][i])
    if x == []:
        rows.append(i)
    else:
        if data_1['Date'][i].startswith('Reported'):
            data_1['Date'][i] = x[0]
        if len(*x) == 11:
            data_1['Date'][i] = x[0][0:7] + x[0][-2:]
        if len(*x) == 9:
            data_1['Date'][i] = x[0]

data_1.drop(rows,inplace = True)


In [9]:
# Añado una nueva columna con los meses a partir de la columna 'Date'

months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}


lst_months = []
for i in data_1['Date']:   
    lst_months.append(months[i[3:6]])
data_1['Month'] = lst_months


In [10]:
# Añado una nueva columna con los dias a partir de la columna 'Date'

list_days = []
for i in data_1.index:
    list_days.append(int(data_1['Date'][i][:2]))
data_1['Day'] = list_days


In [11]:
# Aqui ordeno las columnas

data_1_order = ['Date', 'Day', 'Month', 'Year', 'Country', 'Area', 'Location', 'Type', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species']

data_1 = data_1[data_1_order]

data_1.head()

,Date,Day,Month,Year,Country,Area,Location,Type,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,18-Sep-16,18,9,2016,USA,Florida,"New Smyrna Beach, Volusia County",Unprovoked,Surfing,male,M,16,Minor injury to thigh,N,13h00,Unkown
1,18-Sep-16,18,9,2016,USA,Florida,"New Smyrna Beach, Volusia County",Unprovoked,Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,Unkown
2,18-Sep-16,18,9,2016,USA,Florida,"New Smyrna Beach, Volusia County",Unprovoked,Surfing,male,M,43,Lacerations to lower leg,N,10h43,Unkown
3,17-Sep-16,17,9,2016,AUSTRALIA,Victoria,Thirteenth Beach,Unprovoked,Surfing,Rory Angiolella,M,Unkown,Struck by fin on chest & leg,N,Unkown,Unkown
4,16-Sep-16,16,9,2016,AUSTRALIA,Victoria,Bells Beach,Unprovoked,Surfing,male,M,Unkown,No injury: Knocked off board by shark,N,Unkown,2 m shark


In [12]:
# Me quedo con los paises con mas de 12 ataques de tiburones

lst_delete = []
x = data_1['Country'].value_counts()
for i in data_1.index:
    if  x[data_1['Country'][i]] < 12:
        lst_delete.append(i)
lst_delete

data_1.drop(lst_delete,inplace = True)

data_1['Country'].value_counts()

USA                 859
AUSTRALIA           320
SOUTH AFRICA        117
BRAZIL               47
BAHAMAS              42
NEW ZEALAND          32
REUNION              25
NEW CALEDONIA        20
EGYPT                19
MEXICO               17
FRENCH POLYNESIA     12
Name: Country, dtype: int64

In [13]:
# Creo listas con las coordenadas de los paises
# Situo los valores de forma random alrededor del pais porque no puedo hacerlo con las localizaciones exactas
# Añado nuevas columnas con las coordenadas

dit = {}
dit['USA'] = (190,385) 
dit['AUSTRALIA'] = (820,180)
dit['SOUTH AFRICA'] = (465,160)
dit['BRAZIL'] = (288,240)
dit['BAHAMAS'] = (180,350)
dit['NEW ZEALAND'] = (850,130)
dit['REUNION'] = (550,195)
dit['EGYPT'] = (480,360)
dit['NEW CALEDONIA'] = (860,200)
dit['MEXICO'] = (115,330)
dit['FRENCH POLYNESIA'] = (940,240)


lst1 = []
lst2 = []
lst3 = []
for i in data_1.index:
    x = dit[data_1['Country'][i]][0] + np.random.randint(low = 5, high = 15)
    y = dit[data_1['Country'][i]][1]+ np.random.randint(low = -20, high = 20)
    coord_year = (x, y, data_1['Year'][i])
    coord_month = (x, y, data_1['Month'][i])
    coord = (x,y,0)
    lst1.append(coord_year)
    lst2.append(coord_month) 
    lst3.append(coord)
data_1['Coordinates_year'] = lst1
data_1['Coordinates_month'] = lst2
data_1['Coordinates'] = lst3
    
    

In [15]:
# Exporto estas columnas a un csv para la colocacion de los datos en el mapa.
# Geogebra no me permite exportar una gran cantidad de datos.
 
data_1.to_csv('Partial_Data_Cleaned.csv')
data_1[['Coordinates_year','Coordinates_month','Coordinates']].to_csv('Visualization.csv')

### Los Ataques de tiburones de marzo de 2016


<img src="images/1.png">

### Los ataques de tiburones de mayo de 2015


<img src="images/2.png">